In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

## Section 1: Example Dataset from Lecture Slides

In [2]:
df = pd.read_csv('dataset\Decision Tree Example Car Dataset.csv')

In [3]:
df.columns = [col.lower() for col in df.columns]

unique_label = list(set(df["label"]))

df

,type,color,tyre,doors,label
0,suv,red,white,2,1
1,car,green,black,2,1
2,car,blue,white,2,1
3,suv,green,white,4,1
4,car,red,black,2,1
5,minivan,blue,white,4,0
6,car,green,white,4,0
7,minivan,red,black,4,0
8,suv,green,black,4,0
9,suv,blue,black,2,0


In [4]:
# num_case_dict contains the data showing the population for each label
# ie. we have 30 rows with class label1, 26 rows with label2 and 22 rows with label3
# num_case_dict = { "label1" : 30, "label2": 26, "label3": 22}

def gen_num_case_dict(df):
  # complete your code here
  unique_label = list(set(df["label"]))
  num_case_dict = {i : list(df["label"]).count(i) for i in unique_label}
  return num_case_dict

In [5]:
num_case_dict = gen_num_case_dict(df)
num_case_dict

{0: 9, 1: 5}

In [6]:
assert num_case_dict == {1: 5, 0: 9}

In [7]:
# input num_case_array and return the entropy value

def cal_entropy(num_case_dict):
  # complete your code here
  p = np.array([v / sum(list(num_case_dict.values())) for _ , v in num_case_dict.items()])
  log_arr = [math.log(i , 2) for i in p]
  entropy_sum = -sum(p*log_arr)
  
  return entropy_sum

In [8]:
cal_entropy( num_case_dict )

0.9402859586706309

In [9]:
assert np.allclose(cal_entropy( num_case_dict ), 0.9402859586706311)

In [10]:
# split by color red

df_red = df[ df['color'] == 'red']

display(df_red)

num_case_dict_red = gen_num_case_dict(df_red)

print(num_case_dict_red)

entropy_red = cal_entropy(num_case_dict_red)

print('entropy_red', entropy_red)

entropy_red_weighted = (len(df_red)/len(df)) * entropy_red
entropy_red_weighted


,type,color,tyre,doors,label
0,suv,red,white,2,1
4,car,red,black,2,1
7,minivan,red,black,4,0
10,suv,red,black,2,0


{0: 2, 1: 2}
entropy_red 1.0


0.2857142857142857

In [11]:
# calculate the entropy gain from a decision tree split using a particular feature on a particular df

def cal_entropy_split_gain(df, split_feature):
  # complete your code here

  #Attribute Entropy
  feature = list(df[split_feature])
  feature_unique = list(set(feature))
  df_list = [df[df[split_feature] == i] for i in feature_unique]
  case_list = [gen_num_case_dict(i) for i in df_list]
  sigma = sum([(feature.count(feature_unique[i])/len(feature)) * cal_entropy(case_list[i]) for i in range(len(case_list))])
  
  #Information Gain
  entropy_gain = cal_entropy(num_case_dict) - sigma

  return entropy_gain


In [12]:
cal_entropy_split_gain(df, 'color')

0.029222565658954647

In [13]:
assert np.allclose( cal_entropy_split_gain(df, 'color'), 0.02922256565895487 )

In [14]:
cal_entropy_split_gain(df, 'type')

0.19996253177061096

In [15]:
assert np.allclose( cal_entropy_split_gain(df, 'type'), 0.19996253177061118 )

In [16]:
cal_entropy_split_gain(df, 'doors')

0.15183550136234136

In [17]:
assert np.allclose( cal_entropy_split_gain(df, 'doors'), 0.15183550136234159 )

In [18]:
cal_entropy_split_gain(df, 'tyre')

0.04812703040826927

In [19]:
assert np.allclose( cal_entropy_split_gain(df, 'tyre'), 0.04812703040826949 )

In [20]:
# Given the df and available features for conducting split, find the feature that produces maximum entropy gain.

def choose_best_split(df, available_feature_array):
  # complete your code here
  entropy_dict = {cal_entropy_split_gain(df , i) : i for i in available_feature_array}
  #print("Entropy Dictionary: " , entropy_dict)
  feature_with_max_entropy_gain = entropy_dict[max(list(entropy_dict.keys()))]
  
  return feature_with_max_entropy_gain

In [21]:
columns_list = df.columns.tolist()
columns_list.remove('label')
best_split_feature = choose_best_split(df, columns_list)
best_split_feature

'type'

In [22]:
assert best_split_feature == 'type'

In [23]:
# Generate a dictionary containing the split key and resulting df due to the split key and split feature.

def gen_feature_split_result(df, split_feature):
  # complete your code here
  feature = list(df[split_feature])
  feature_unique = list(set(feature))
  df_split_dict = {i : df[df[split_feature] == i] for i in feature_unique}

  return df_split_dict

In [24]:
df_split_dict = gen_feature_split_result(df, best_split_feature)
df_split_dict

{'car':    type  color   tyre  doors  label
 1   car  green  black      2      1
 2   car   blue  white      2      1
 4   car    red  black      2      1
 6   car  green  white      4      0
 11  car   blue  black      4      0,
 'minivan':        type  color   tyre  doors  label
 5   minivan   blue  white      4      0
 7   minivan    red  black      4      0
 13  minivan  green  white      4      0,
 'suv':    type  color   tyre  doors  label
 0   suv    red  white      2      1
 3   suv  green  white      4      1
 8   suv  green  black      4      0
 9   suv   blue  black      2      0
 10  suv    red  black      2      0
 12  suv  green  black      2      0}

In [25]:
assert len(df_split_dict) == 3
assert len(df_split_dict['suv']) == 6
assert len(df_split_dict['minivan']) == 3
assert len(df_split_dict['car']) == 5

In [26]:
df_split_dict['suv']

,type,color,tyre,doors,label
0,suv,red,white,2,1
3,suv,green,white,4,1
8,suv,green,black,4,0
9,suv,blue,black,2,0
10,suv,red,black,2,0
12,suv,green,black,2,0


In [27]:
df_split_dict['minivan']

,type,color,tyre,doors,label
5,minivan,blue,white,4,0
7,minivan,red,black,4,0
13,minivan,green,white,4,0


In [28]:
df_split_dict['car']

,type,color,tyre,doors,label
1,car,green,black,2,1
2,car,blue,white,2,1
4,car,red,black,2,1
6,car,green,white,4,0
11,car,blue,black,4,0


## Section 2: Create Tree Structure

In [29]:
class Node:
  def __init__(self, df, parent, parent_edge, parent_split_feature, available_features):
    self.df = df
    self.parent = parent
    self.parent_edge = parent_edge
    self.children = {}
    self.split_feature = None
    self.available_features = available_features
    self.explored = False
    self.predicted_label = None
    if parent is not None:
      parent.split_feature = parent_split_feature

  def add_child(self, split_key, new_node):
    self.children[split_key] = new_node

In [30]:
class TreeGraph:
  def __init__(self, df, available_features):
    self.root = Node(df, None, None, None, available_features)
    self.nodes = []
    self.nodes.append(self.root)

  def add_node(self, df, parent_node, parent_edge, parent_split_feature, available_features):
    new_node = Node(df, parent_node, parent_edge, parent_split_feature, available_features)
    self.nodes.append(new_node)
    if parent_node is not None:
      parent_node.add_child(  parent_edge, new_node )

  def gen_decision_tree(self):
    # Complete this code here
    # When you run this function it should generate the full decision tree by creating all the nodes for the tree

    i = 0
    complete = False
    while not complete:
      
      cur_node = self.nodes[i]
      
      #Check if target is certain yet or find mode at the leaf
      if (len(cur_node.df["label"].value_counts()) == 1 or len(cur_node.available_features) == 0) and not complete:
    
        i += 1

        if i < len(self.nodes):
          cur_node = self.nodes[i]
        else:
          complete = True
      #Add system if there is no next node it will vote on the current node

      #Adding Nodes
      elif not complete:


        #Finding Best split
        cur_node.split_feature = choose_best_split(cur_node.df , cur_node.available_features)

        #print("Split feature: " , cur_node.split_feature)
        #print("Node Index: " , self.nodes.index(self.nodes[i]))
        #print("Dataframe: \n" , cur_node.df)

        #Remove the used feature
        new_available_features = cur_node.available_features.copy()
        new_available_features.remove(cur_node.split_feature)

        
        #Setting Children
        prep_child_dict = gen_feature_split_result(cur_node.df , cur_node.split_feature)
        #print("Prep for child dict key: \n" , prep_child_dict.keys())
  
        for edge , df in prep_child_dict.items():
          self.add_node(df , cur_node , edge , cur_node.split_feature , new_available_features) 
          #print("Node Added: " , edge)
        
        i += 1

    return

In [31]:
type(df.columns)

pandas.core.indexes.base.Index

In [32]:
columns_list = df.columns.tolist()
columns_list.remove('label')
g = TreeGraph(df, columns_list)
g.gen_decision_tree()
g.nodes

In [33]:
print(g.root.children)

{'car': <__main__.Node object at 0x000002833EC2D1E0>, 'minivan': <__main__.Node object at 0x00000283408AF5B0>, 'suv': <__main__.Node object at 0x00000283408AF460>}


In [34]:
assert len(g.nodes) == 8

In [35]:
assert g.root.split_feature == 'type'

In [36]:
assert len(g.root.children) == 3

In [37]:
g.root.children.items()

dict_items([('car', <__main__.Node object at 0x000002833EC2D1E0>), ('minivan', <__main__.Node object at 0x00000283408AF5B0>), ('suv', <__main__.Node object at 0x00000283408AF460>)])

In [38]:
for node in g.nodes:
  if node.split_feature == 'doors':
    test_case = node.df
    display(test_case)

assert len(test_case) == 5
assert np.array_equal( test_case['color'].to_numpy() , np.array(['green', 'blue', 'red', 'green', 'blue'], dtype=object) )

,type,color,tyre,doors,label
1,car,green,black,2,1
2,car,blue,white,2,1
4,car,red,black,2,1
6,car,green,white,4,0
11,car,blue,black,4,0


In [39]:
for node in g.nodes:
  if node.split_feature == 'tyre':
    test_case = node.df
    display(test_case)

assert len(test_case) == 6
assert np.array_equal( test_case['color'].to_numpy() , np.array(['red', 'green', 'green', 'blue', 'red', 'green'], dtype=object) )

,type,color,tyre,doors,label
0,suv,red,white,2,1
3,suv,green,white,4,1
8,suv,green,black,4,0
9,suv,blue,black,2,0
10,suv,red,black,2,0
12,suv,green,black,2,0


In [40]:
for node in g.nodes:
  if node.split_feature == 'type':
    test_case = node.df
    display(test_case)

assert len(test_case) == 14
assert np.array_equal( test_case['color'].to_numpy() , np.array(['red', 'green', 'blue', 'green', 'red', 'blue', 'green', 'red', 'green', 'blue', 'red', 'blue', 'green', 'green'], dtype=object) )

,type,color,tyre,doors,label
0,suv,red,white,2,1
1,car,green,black,2,1
2,car,blue,white,2,1
3,suv,green,white,4,1
4,car,red,black,2,1
5,minivan,blue,white,4,0
6,car,green,white,4,0
7,minivan,red,black,4,0
8,suv,green,black,4,0
9,suv,blue,black,2,0


## Section 3: Do Inference to Make Predictions

In [41]:
# Now let's do inference with the tree we've generated

In [42]:
df

,type,color,tyre,doors,label
0,suv,red,white,2,1
1,car,green,black,2,1
2,car,blue,white,2,1
3,suv,green,white,4,1
4,car,red,black,2,1
5,minivan,blue,white,4,0
6,car,green,white,4,0
7,minivan,red,black,4,0
8,suv,green,black,4,0
9,suv,blue,black,2,0


In [43]:
import copy

df_prediction = df.copy()
df_prediction['prediction'] = None

In [44]:
df_prediction

,type,color,tyre,doors,label,prediction
0,suv,red,white,2,1,None
1,car,green,black,2,1,None
2,car,blue,white,2,1,None
3,suv,green,white,4,1,None
4,car,red,black,2,1,None
5,minivan,blue,white,4,0,None
6,car,green,white,4,0,None
7,minivan,red,black,4,0,None
8,suv,green,black,4,0,None
9,suv,blue,black,2,0,None


In [45]:
# This function should fill in the predicted values using our decision tree into the df in the column 'prediction'

def generate_prediction(df_prediction, treeGraph):
  # Complete the code here

  prediction = []
  
  for i in df_prediction.index.values:
  
    #Starting at root node
    node = treeGraph.root
    predicted = False

    #Setting up row
    df_prediction_row = df_prediction.loc[i]
    #print("Current Row: \n" , df_prediction_row)

    #Loop until row is predicted
    while not predicted:
      if node.split_feature != None and df_prediction_row[node.split_feature] in node.children:
        #print("Node's Children: " , node.children)
        node = node.children[df_prediction_row[node.split_feature]]
      elif node.split_feature == None or df_prediction.at[i , node.split_feature] not in node.children:        
        prediction.append(node.df["label"].value_counts().idxmax())
        #print("Predicted: " , predicted)
        predicted = True

  df_prediction["prediction"] = prediction

  return df_prediction

In [46]:
g.nodes[2].df

,type,color,tyre,doors,label
5,minivan,blue,white,4,0
7,minivan,red,black,4,0
13,minivan,green,white,4,0


In [47]:
prediction_result_df = generate_prediction(df_prediction,g)
prediction_result_df

,type,color,tyre,doors,label,prediction
0,suv,red,white,2,1,1
1,car,green,black,2,1,1
2,car,blue,white,2,1,1
3,suv,green,white,4,1,1
4,car,red,black,2,1,1
5,minivan,blue,white,4,0,0
6,car,green,white,4,0,0
7,minivan,red,black,4,0,0
8,suv,green,black,4,0,0
9,suv,blue,black,2,0,0


In [48]:
assert np.array_equal( prediction_result_df['prediction'].to_numpy(), np.array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=object) )

In [49]:
def prediction_accuracy(prediction_result_df):
  correct = 0
  for index, row in prediction_result_df.iterrows():
    if row['label'] == row['prediction']:
      correct = correct + 1
  accuracy = correct / len(prediction_result_df)
  return accuracy

In [50]:
prediction_accuracy_result = prediction_accuracy(prediction_result_df)
prediction_accuracy_result

1.0

In [51]:
assert prediction_accuracy_result == 1

## Section 4: Test with Proper Dataset

In [52]:
# Get dataset from here: https://archive.ics.uci.edu/dataset/19/car+evaluation

# buying - buying price
# maintainance - price of the maintenance
# doors - number of doors
# person - capacity in terms of persons to carry
# luggage_boot - the size of luggage boot
# safety - estimated safety of the car
# label - evaulation level (unacceptable, acceptable, good, very good)

df_car = pd.read_csv('dataset\car.data', sep=',', header=None)

# Assign column names to the DataFrame
df_car.columns = ['buying', 'maintainance', 'doors', 'person', 'luggage_boot', 'safety', 'label' ]

df_car.head()

,buying,maintainance,doors,person,luggage_boot,safety,label
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [53]:
from sklearn.model_selection import train_test_split

df_car_train, df_car_test = train_test_split(df_car, test_size=0.2, random_state=42)

In [54]:
df_car_train

,buying,maintainance,doors,person,luggage_boot,safety,label
107,vhigh,vhigh,5more,more,big,high,unacc
901,med,vhigh,3,4,small,med,unacc
1709,low,low,5more,2,big,high,unacc
706,high,med,4,2,med,med,unacc
678,high,med,3,2,med,low,unacc
...,...,...,...,...,...,...,...
1130,med,med,3,more,med,high,vgood
1294,med,low,5more,more,big,med,good
860,high,low,5more,more,med,high,acc
1459,low,high,4,2,small,med,unacc


In [55]:
columns_list = df_car.columns.tolist()
columns_list.remove('label')
g_car = TreeGraph(df_car_train, columns_list)
g_car.gen_decision_tree()

In [56]:
columns_list

['buying', 'maintainance', 'doors', 'person', 'luggage_boot', 'safety']

In [57]:
len(g_car.nodes)

346

In [58]:
df_car_prediction = df_car_test.copy()
df_car_prediction['prediction'] = None
df_car_prediction_result = generate_prediction(df_car_prediction, g_car)

In [59]:
df_car_prediction_result['prediction']

599     unacc
1201     good
628     unacc
1498      acc
1263    unacc
        ...  
100     unacc
274     unacc
1206    unacc
101     unacc
1084    unacc
Name: prediction, Length: 346, dtype: object

In [60]:
df_car_prediction_result

,buying,maintainance,doors,person,luggage_boot,safety,label,prediction
599,high,high,4,2,med,high,unacc,unacc
1201,med,low,2,4,med,med,acc,good
628,high,high,5more,2,big,med,unacc,unacc
1498,low,high,5more,4,med,med,acc,acc
1263,med,low,4,more,med,low,unacc,unacc
...,...,...,...,...,...,...,...,...
100,vhigh,vhigh,5more,more,small,med,unacc,unacc
274,vhigh,med,4,2,med,med,unacc,unacc
1206,med,low,2,more,small,low,unacc,unacc
101,vhigh,vhigh,5more,more,small,high,unacc,unacc


In [61]:
prediction_accuracy_result = prediction_accuracy(df_car_prediction_result)
prediction_accuracy_result

0.9075144508670521

In [62]:
assert prediction_accuracy_result > 0.9

In [63]:
assert len(df_car_prediction_result[df_car_prediction_result['prediction'].isna()]) == 0